# 2. Transformación y Limpieza de Datos

Este notebook aplica transformaciones y limpieza a los datos consolidados usando funciones modulares del paquete `src/cleaning.py`.

## Pipeline de transformación

1. **Carga** de datos consolidados
2. **Limpieza** de formato numérico (europeo → estándar)
3. **Separación** de valores absolutos y porcentajes
4. **Pivot** de formato largo a ancho
5. **Renombrado** de columnas para claridad
6. **Verificación** final y guardado

---

**Input:** 
- `data/cleanedDataset/gastos_consolidado.csv` (generado por notebook 1)
- `data/cleanedDataset/fondos_consolidado.csv` (generado por notebook 1)

**Output:** 
- `data/cleanedDataset/gastos_clean.csv`
- `data/cleanedDataset/fondos_clean.csv`

In [38]:
# Importar librerías estándar
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración de pandas
pd.options.display.float_format = '{:.2f}'.format

# Importar módulo de limpieza personalizado
import sys
sys.path.append('../src')
from cleaning import (
    limpiar_columna_total,
    separar_valores_porcentajes,
    pivotear_dataset,
    renombrar_columnas_gastos,
    renombrar_columnas_fondos,
    preparar_para_indice
)

print("✅ Librerías y módulos importados correctamente")

✅ Librerías y módulos importados correctamente


## 6. Limpieza y Transformación de Datos

En base a los problemas identificados en la exploración (Notebook 1), procedemos a limpiar y transformar los datos para hacerlos aptos para el análisis numérico y temporal.

### Carga de datos consolidados

In [39]:
# Cargar datasets consolidados del notebook anterior
df_gastos = pd.read_csv('../data/consolidatedDataset/gastos_consolidados.csv')
df_fondos = pd.read_csv('../data/consolidatedDataset/fondos_consolidados.csv')

print("="*70)
print("DATASETS CONSOLIDADOS CARGADOS")
print("="*70)
print(f"\n📊 Gastos I+D (2000-2024):")
print(f"   Dimensiones: {df_gastos.shape}")
print(f"   Columnas: {list(df_gastos.columns)}")
print(f"   Tipo 'Total': {df_gastos['Total'].dtype}")
print(f"   Rango años: {df_gastos['Años'].min():.0f} - {df_gastos['Años'].max():.0f}")

print(f"\n📊 Fondos I+D (2000-2024):")
print(f"   Dimensiones: {df_fondos.shape}")
print(f"   Columnas: {list(df_fondos.columns)}")
print(f"   Tipo 'Total': {df_fondos['Total'].dtype}")
print(f"   Rango años: {df_fondos['Años'].min():.0f} - {df_fondos['Años'].max():.0f}")

# Mostrar ejemplo de los datos sin limpiar
print("\n📋 Ejemplo de datos SIN limpiar:")
print(df_gastos.head(10))

DATASETS CONSOLIDADOS CARGADOS

📊 Gastos I+D (2000-2024):
   Dimensiones: (225, 3)
   Columnas: ['Sectores/unidad', 'Años', 'Total']
   Tipo 'Total': object
   Rango años: 2000 - 2024

📊 Fondos I+D (2000-2024):
   Dimensiones: (225, 3)
   Columnas: ['Sectores/unidad', 'Años', 'Total']
   Tipo 'Total': object
   Rango años: 2000 - 2024

📋 Ejemplo de datos SIN limpiar:
          Sectores/unidad    Años         Total
0  Total (miles de euros) 2000.00   5.718.988,3
1  Total (miles de euros) 2001.00   6.227.157,1
2  Total (miles de euros) 2002.00     7.193.538
3  Total (miles de euros) 2003.00   8.213.035,6
4  Total (miles de euros) 2004.00   8.945.760,7
5  Total (miles de euros) 2005.00    10.196.871
6  Total (miles de euros) 2006.00  11.815.217,9
7  Total (miles de euros) 2007.00  13.342.370,6
8  Total (miles de euros) 2008.00  14.701.392,9
9  Total (miles de euros) 2009.00  14.581.675,7


### 6.1 Limpieza de la columna 'Total'

**Problema identificado:** 
- La columna 'Total' está en formato texto con notación europea
- Ejemplo: "1.234.567,89" en lugar de 1234567.89
- Valores faltantes representados como ".."

**Solución aplicada:**
- Reemplazar ".." por NaN
- Eliminar puntos (separador de miles)
- Reemplazar comas por puntos (separador decimal)
- Convertir a tipo numérico (float)

Utilizamos la función `limpiar_columna_total()` del módulo `cleaning.py`

In [40]:
# Aplicar limpieza de columna Total usando función modular
df_gastos_limpio = limpiar_columna_total(df_gastos, "Gastos I+D")
df_fondos_limpio = limpiar_columna_total(df_fondos, "Fondos I+D")

print("\n✅ Limpieza completada")
print(f"   Tipo 'Total' en gastos: {df_gastos_limpio['Total'].dtype}")
print(f"   Tipo 'Total' en fondos: {df_fondos_limpio['Total'].dtype}")


🧹 Limpiando columna 'Total' de Gastos I+D...
   Tipo original: object
   Tipo final: float64
   Valores NaN: 0

🧹 Limpiando columna 'Total' de Fondos I+D...
   Tipo original: object
   Tipo final: float64
   Valores NaN: 0

✅ Limpieza completada
   Tipo 'Total' en gastos: float64
   Tipo 'Total' en fondos: float64


### 6.2 Separación de valores absolutos y porcentajes

**Problema identificado:**
- Los datasets mezclan valores absolutos (miles de €) con porcentajes
- Ejemplo: "Total (miles de euros)" y "Empresas: %"

**Solución aplicada:**
- Filtrar solo las filas con valores absolutos
- Eliminar las filas que contienen "%"

Utilizamos la función `separar_valores_porcentajes()` del módulo `cleaning.py`

In [41]:
# Separar valores absolutos de porcentajes
df_gastos_valores = separar_valores_porcentajes(df_gastos_limpio)
df_fondos_valores = separar_valores_porcentajes(df_fondos_limpio)

print("\n📋 Verificación de separación:")
print(f"   Gastos - Categorías únicas:")
print(f"   {df_gastos_valores['Sectores/unidad'].unique()}")
print(f"\n   Fondos - Categorías únicas:")
print(f"   {df_fondos_valores['Sectores/unidad'].unique()}")


📊 Separando valores absolutos de porcentajes...
   Filas con porcentajes: 100
   Filas con valores absolutos: 125

📊 Separando valores absolutos de porcentajes...
   Filas con porcentajes: 100
   Filas con valores absolutos: 125

📋 Verificación de separación:
   Gastos - Categorías únicas:
   ['Total (miles de euros)' 'Administración Pública: Total (miles de euros)'
 'Enseñanza Superior: Total (miles de euros)'
 'Empresas: Total (miles de euros)' 'IPSFL: Total (miles de euros)']

   Fondos - Categorías únicas:
   ['Total (miles de euros)' 'Administración Pública: Total (miles de euros)'
 'Empresas: Total (miles de euros)' 'IPSFL: Total (miles de euros)'
 'Resto del Mundo: Total (miles de euros)']


### 6.3 Transformación de formato largo a ancho (Pivot)

**Problema identificado:**
- Los datos están en formato largo (múltiples filas por año)
- Dificulta el análisis temporal

**Formato largo (actual):**
Años | Sectores/unidad | Total 2000 | Total (miles de euros) | 5718919 2000 | Empresas: Total (miles de euros) | 3456789 2001 | Total (miles de euros) | 6123456

**Formato ancho (objetivo):**
Años | Total | Admin_Publica | Empresas | ... 2000 | 5718919 | 1234567 | 3456789 | ... 2001 | 6123456 | 1345678 | 3621011 | ...

Utilizamos la función `pivotear_dataset()` del módulo `cleaning.py`

In [42]:
# Transformar de formato largo a ancho
df_gastos_wide = pivotear_dataset(df_gastos_valores, "Gastos I+D")
df_fondos_wide = pivotear_dataset(df_fondos_valores, "Fondos I+D")

print("\n📋 Primeras filas del dataset pivotado (gastos):")
print(df_gastos_wide.head())

print("\n📋 Primeras filas del dataset pivotado (fondos):")
print(df_fondos_wide.head())


🔄 Transformando Gastos I+D de formato largo a ancho...
   Dimensiones originales: (125, 3)
   Dimensiones después de pivot: (25, 6)
   Columnas creadas: ['Años', 'Administración Pública: Total (miles de euros)', 'Empresas: Total (miles de euros)', 'Enseñanza Superior: Total (miles de euros)', 'IPSFL: Total (miles de euros)', 'Total (miles de euros)']

🔄 Transformando Fondos I+D de formato largo a ancho...
   Dimensiones originales: (125, 3)
   Dimensiones después de pivot: (25, 6)
   Columnas creadas: ['Años', 'Administración Pública: Total (miles de euros)', 'Empresas: Total (miles de euros)', 'IPSFL: Total (miles de euros)', 'Resto del Mundo: Total (miles de euros)', 'Total (miles de euros)']

📋 Primeras filas del dataset pivotado (gastos):
Sectores/unidad    Años  Administración Pública: Total (miles de euros)  \
0               2000.00                                       904776.10   
1               2001.00                                       989011.10   
2               2002.

### 6.4 Renombrado de columnas

**Problema identificado:**
- Nombres de columnas muy largos y difíciles de usar
- Ejemplo: "Administración Pública: Total (miles de euros)"

**Solución aplicada:**
- Simplificar nombres para mayor claridad
- Ejemplo: "Admin_Publica", "Enseñanza_Superior", "Empresas", etc.

Utilizamos las funciones `renombrar_columnas_gastos()` y `renombrar_columnas_fondos()` del módulo `cleaning.py`

In [43]:
# Renombrar columnas para mayor claridad
df_gastos_wide = renombrar_columnas_gastos(df_gastos_wide)
df_fondos_wide = renombrar_columnas_fondos(df_fondos_wide)

print("\n📋 Columnas finales:")
print(f"   Gastos: {list(df_gastos_wide.columns)}")
print(f"   Fondos: {list(df_fondos_wide.columns)}")


✏️  Columnas renombradas:
   'Total (miles de euros)' → 'Total'
   'Administración Pública: Total (miles de euros)' → 'Admin_Publica'
   'Enseñanza Superior: Total (miles de euros)' → 'Enseñanza_Superior'
   'Empresas: Total (miles de euros)' → 'Empresas'
   'IPSFL: Total (miles de euros)' → 'IPSFL'

✏️  Columnas renombradas:
   'Total (miles de euros)' → 'Total'
   'Administración Pública: Total (miles de euros)' → 'Admin_Publica'
   'Empresas: Total (miles de euros)' → 'Empresas'
   'IPSFL: Total (miles de euros)' → 'IPSFL'
   'Resto del Mundo: Total (miles de euros)' → 'Resto_Mundo'

📋 Columnas finales:
   Gastos: ['Años', 'Admin_Publica', 'Empresas', 'Enseñanza_Superior', 'IPSFL', 'Total']
   Fondos: ['Años', 'Admin_Publica', 'Empresas', 'IPSFL', 'Resto_Mundo', 'Total']


### 6.5 Preparación del índice

**Transformación final:**
- Establecer 'Años' como índice del DataFrame
- Facilita el análisis temporal y selección de datos

Utilizamos la función `preparar_para_indice()` del módulo `cleaning.py`

In [44]:
# Establecer Años como índice para facilitar análisis temporal
df_gastos_wide = preparar_para_indice(df_gastos_wide)
df_fondos_wide = preparar_para_indice(df_fondos_wide)

print("✅ Índice establecido correctamente")


🔑 'Años' establecido como índice
   Índice: 2000 - 2024

🔑 'Años' establecido como índice
   Índice: 2000 - 2024
✅ Índice establecido correctamente


### 6.6 Verificación final de los datos limpios

Revisión completa de los datasets transformados:
- Estructura y dimensiones
- Tipos de datos
- Valores faltantes
- Estadísticas descriptivas

In [45]:
print("="*70)
print("VERIFICACIÓN FINAL DE DATASETS LIMPIOS")
print("="*70)

print("\n📊 DATASET: GASTOS I+D")
print("="*70)
print(f"Dimensiones: {df_gastos_wide.shape}")
print(f"Índice: {df_gastos_wide.index.min():.0f} - {df_gastos_wide.index.max():.0f}")
print(f"\nColumnas y tipos:")
print(df_gastos_wide.dtypes)
print(f"\nValores NaN por columna:")
print(df_gastos_wide.isnull().sum())
print(f"\nEstadísticas descriptivas:")
print(df_gastos_wide.describe())

print("\n" + "="*70)
print("📊 DATASET: FONDOS I+D")
print("="*70)
print(f"Dimensiones: {df_fondos_wide.shape}")
print(f"Índice: {df_fondos_wide.index.min():.0f} - {df_fondos_wide.index.max():.0f}")
print(f"\nColumnas y tipos:")
print(df_fondos_wide.dtypes)
print(f"\nValores NaN por columna:")
print(df_fondos_wide.isnull().sum())
print(f"\nEstadísticas descriptivas:")
print(df_fondos_wide.describe())

print("\n" + "="*70)
print("✅ VERIFICACIÓN COMPLETADA")
print("="*70)
print("   ✅ Formato numérico correcto (float64)")
print("   ✅ Estructura adecuada para análisis temporal")
print("   ✅ Nombres de columnas claros")
print("   ✅ Años como índice")

total_nan = df_gastos_wide.isnull().sum().sum() + df_fondos_wide.isnull().sum().sum()
if total_nan > 0:
    print(f"   ⚠️  {total_nan} valores NaN (datos no disponibles del INE) - se mantendrán")
else:
    print("   ✅ Sin valores NaN")


VERIFICACIÓN FINAL DE DATASETS LIMPIOS

📊 DATASET: GASTOS I+D
Dimensiones: (25, 5)
Índice: 2000 - 2024

Columnas y tipos:
Sectores/unidad
Admin_Publica         float64
Empresas              float64
Enseñanza_Superior    float64
IPSFL                 float64
Total                 float64
dtype: object

Valores NaN por columna:
Sectores/unidad
Admin_Publica         0
Empresas              0
Enseñanza_Superior    0
IPSFL                 0
Total                 0
dtype: int64

Estadísticas descriptivas:
Sectores/unidad  Admin_Publica    Empresas  Enseñanza_Superior    IPSFL  \
count                    25.00       25.00               25.00    25.00   
mean                2424949.90  7386992.03          3698336.16 33636.06   
std                  868648.14  2532555.50          1033414.94 16600.27   
min                  904776.10  3068994.40          1693881.60 11673.70   
25%                 1970823.50  6557529.00          3265738.50 21551.10   
50%                 2515228.40  7396369.00   

### 6.7 Guardado de datasets limpios

Los datos limpios se guardan en formato CSV para ser utilizados en el siguiente notebook de análisis y visualización.

In [46]:
# ============================================================================
# GUARDAR DATASETS LIMPIOS
# ============================================================================

print("\n" + "="*70)
print("GUARDANDO DATASETS LIMPIOS")
print("="*70)

# Guardar datasets en formato wide con índice de años
df_gastos_wide.to_csv('../data/cleanedDataset/gastos_clean.csv')
df_fondos_wide.to_csv('../data/cleanedDataset/fondos_clean.csv')

print("\n✅ Datos limpios guardados exitosamente:")
print(f"   📁 ../data/cleanedDataset/gastos_clean.csv")
print(f"      Dimensiones: {df_gastos_wide.shape}")
print(f"   📁 ../data/cleanedDataset/fondos_clean.csv")
print(f"      Dimensiones: {df_fondos_wide.shape}")

print("\n" + "="*70)
print("📌 PRÓXIMO PASO")
print("="*70)
print("   Ejecutar el notebook eda.ipynb para:")
print("   - Análisis visual de tendencias temporales")
print("   - Distribución sectorial del gasto")
print("   - Impacto de eventos económicos")
print("   - Conclusiones del análisis")


GUARDANDO DATASETS LIMPIOS

✅ Datos limpios guardados exitosamente:
   📁 ../data/cleanedDataset/gastos_clean.csv
      Dimensiones: (25, 5)
   📁 ../data/cleanedDataset/fondos_clean.csv
      Dimensiones: (25, 5)

📌 PRÓXIMO PASO
   Ejecutar el notebook eda.ipynb para:
   - Análisis visual de tendencias temporales
   - Distribución sectorial del gasto
   - Impacto de eventos económicos
   - Conclusiones del análisis
